In [ ]:
# ===================================================================
# Cell 1: 데이터 로드 + GPU 설정
# ===================================================================
import numpy as np
import gc
import tensorflow as tf
import psutil

print("GPU 메모리 동적 할당 설정 중...\n")
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("[완료] GPU 메모리 동적 할당 설정 완료!\n")
    except RuntimeError as e:
        print(f"[경고] GPU 설정 경고: {e}\n")

print("데이터 로드 중...\n")
DATA_PATH = '/kaggle/input/xy-data'
X_train = np.load(f'{DATA_PATH}/X_train.npy')
y_train = np.load(f'{DATA_PATH}/y_train.npy')
X_val = np.load(f'{DATA_PATH}/X_val.npy')
y_val = np.load(f'{DATA_PATH}/y_val.npy')

print("[완료] 로드 완료!")
print(f"   X_train: {X_train.shape}")
print(f"   y_train: {y_train.shape}")
print(f"   X_val: {X_val.shape}")
print(f"   y_val: {y_val.shape}")

IMG_SIZE = X_train.shape[1]
print(f"\n[확인] IMG_SIZE: {IMG_SIZE}")
print(f"   데이터 메모리: {(X_train.nbytes + X_val.nbytes) / (1024**3):.2f} GB")

ram = psutil.virtual_memory()
print(f"\n[메모리] RAM: {ram.used/1024**3:.1f}GB / {ram.total/1024**3:.1f}GB ({ram.percent:.1f}%)")

print("\n[목표] MobileNetV2 극한 개선으로 77.5-78% 달성!")
print("="*60)

In [ ]:
# ===================================================================
# Cell 2: 모델 정의 (검증된 설정 + 약간 개선)
# ===================================================================
print("🔥 검증된 설정으로 모델 구축...\n")

from tensorflow import keras
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam

gc.collect()

# ===================================================================
# 1. Focal Loss (원래 성공했던 설정!)
# ===================================================================
def focal_loss(gamma=1.5, alpha=0.5):  # 원래대로!
    def focal_loss_fixed(y_true, y_pred):
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1.0 - epsilon)
        
        y_true = tf.cast(y_true, tf.float32)
        p_t = tf.where(tf.equal(y_true, 1), y_pred, 1 - y_pred)
        alpha_t = tf.where(tf.equal(y_true, 1), alpha, 1 - alpha)
        focal_weight = tf.pow(1.0 - p_t, gamma)
        focal_loss = -alpha_t * focal_weight * tf.math.log(p_t)
        
        return tf.reduce_mean(focal_loss)
    
    return focal_loss_fixed

print("[설정] Focal Loss: gamma=1.5 (검증됨), alpha=0.5\n")

# ===================================================================
# 2. 데이터 증강 (원래 설정 + 약간만 추가)
# ===================================================================
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),    # 원래대로
    layers.RandomZoom(0.1),        # 원래대로
    layers.RandomContrast(0.1),    # 이것만 추가
], name='data_augmentation')

print("[설정] 데이터 증강 (검증된 설정):")
print("      - RandomFlip")
print("      - RandomRotation: 0.1")
print("      - RandomZoom: 0.1")
print("      - RandomContrast: 0.1 (추가)\n")

# ===================================================================
# 3. MobileNetV2
# ===================================================================
print("MobileNetV2 로딩...\n")

base_model = MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights='imagenet'
)

print(f"[완료] MobileNetV2 로드!")
print(f"      전체 layers: {len(base_model.layers)}\n")

# ===================================================================
# 4. Fine-tuning (원래 설정 + 약간만 증가)
# ===================================================================
FINE_TUNE_LAYERS = 50  # 원래는 40, 50으로 약간만 증가

print(f"[설정] Fine-tuning:")
print(f"      - Freeze: 처음 {len(base_model.layers) - FINE_TUNE_LAYERS} layers")
print(f"      - Fine-tune: 마지막 {FINE_TUNE_LAYERS} layers\n")

for layer in base_model.layers[:-FINE_TUNE_LAYERS]:
    layer.trainable = False
    
for layer in base_model.layers[-FINE_TUNE_LAYERS:]:
    layer.trainable = True

# ===================================================================
# 5. 모델 구축 (원래 설정 그대로)
# ===================================================================
model = models.Sequential([
    layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3)),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid', dtype='float32')
])

print("[완료] 모델 구축 (검증된 구조)\n")

# ===================================================================
# 6. 컴파일 (원래 설정 그대로)
# ===================================================================
model.compile(
    optimizer=Adam(learning_rate=0.0005),  # 원래대로
    loss=focal_loss(gamma=1.5, alpha=0.5),
    metrics=['accuracy', 
             tf.keras.metrics.AUC(name='auc'),
             tf.keras.metrics.Precision(name='precision'),
             tf.keras.metrics.Recall(name='recall')]
)

print("="*60)
print("📊 모델 요약")
print("="*60)
model.summary()

print("\n" + "="*60)
print("✅ 검증된 모델 준비 완료!")
print("="*60)
print(f"모델: MobileNetV2")
print(f"손실: Focal Loss (gamma=1.5)")
print(f"LR: 0.0005")
print(f"Fine-tuning: {FINE_TUNE_LAYERS} layers")
print(f"증강: 4개 (Contrast 추가)")
print("="*60)

In [ ]:
# ===================================================================
# Cell 3: 학습 (검증된 설정 + Epochs만 증가)
# ===================================================================
print("\n🚀 학습 시작!")
print("="*60)

from sklearn.utils import class_weight
import pickle

gc.collect()

class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}
print(f"✅ Class weights: {class_weight_dict}\n")

# Dataset 생성
print("Dataset 생성...\n")

BATCH_SIZE = 10

def augment_data(image, label):
    image = data_augmentation(image, training=True)
    return image, label

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=10000, seed=42)
train_dataset = train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.map(augment_data, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_dataset = val_dataset.batch(BATCH_SIZE)
val_dataset = val_dataset.prefetch(tf.data.AUTOTUNE)

print(f"[완료] Dataset 준비!")
print(f"   Train: {len(y_train):,} samples")
print(f"   Val: {len(y_val):,} samples\n")

del X_train
gc.collect()
print("[완료] 메모리 정리\n")

# Callbacks
checkpoint = callbacks.ModelCheckpoint(
    'best_model_improved.weights.h5',
    monitor='val_accuracy',
    save_best_only=True,
    save_weights_only=True,
    mode='max',
    verbose=1
)

early_stop = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,  # 원래대로
    restore_best_weights=True,
    verbose=1
)

reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,  # 원래대로
    min_lr=1e-7,
    verbose=1
)

EPOCHS = 35  # 25 → 35 (이것만 증가)

print("="*60)
print("🔥 학습 설정")
print("="*60)
print(f"Epochs: {EPOCHS} (25→35 증가)")
print(f"Batch: {BATCH_SIZE}")
print(f"LR: 0.0005")
print(f"Focal Loss: gamma=1.5 (검증됨)")
print(f"목표: 76.5-77.5%")
print("="*60)

try:
    history = model.fit(
        train_dataset,
        epochs=EPOCHS,
        validation_data=val_dataset,
        class_weight=class_weight_dict,
        callbacks=[checkpoint, early_stop, reduce_lr],
        verbose=1
    )
    
    print("\n" + "="*60)
    print("✅ 학습 완료!")
    print("="*60)
    
    best_val_acc = max(history.history['val_accuracy'])
    best_epoch = history.history['val_accuracy'].index(best_val_acc) + 1
    
    print(f"\n📊 최종 결과:")
    print(f"   최고 정확도: {best_val_acc*100:.2f}%")
    print(f"   달성 Epoch: {best_epoch}/{EPOCHS}")
    
    improvement = (best_val_acc - 0.7635) * 100
    print(f"\n🎯 개선:")
    print(f"   이전: 76.35%")
    print(f"   현재: {best_val_acc*100:.2f}%")
    print(f"   향상: {improvement:+.2f}%p")
    
    if best_val_acc >= 0.77:
        print(f"\n🎉 목표 달성!")
    elif best_val_acc >= 0.765:
        print(f"\n✅ 개선 성공!")
    else:
        print(f"\n📊 약간 개선")
    
    # 저장
    print("\n💾 저장 중...")
    with open('history_improved.pkl', 'wb') as f:
        pickle.dump(history.history, f)
    
    model.save('final_mobilenet_improved.keras')
    
    print("\n✅ 저장 완료:")
    print("   - history_improved.pkl")
    print("   - final_mobilenet_improved.keras")
    print("   - best_model_improved.weights.h5")
    
    import os
    size = os.path.getsize('final_mobilenet_improved.keras') / (1024**2)
    print(f"\n📦 크기: {size:.1f} MB")
    
    print("\n" + "="*60)
    print("🚨 Save Version 하세요!")
    print("="*60)
    print("Version name: 'mobilenet-improved-77pct'")
    print("="*60)

except Exception as e:
    print(f"\n❌ ERROR: {e}")
    import traceback
    traceback.print_exc()